#### Don't forget to delete the hdmi_out and hdmi_in when finished

# Lines Filter Example
In this notebook, we will create a box formed of four lines onto our image. The box is created by checking the pixel locations of each pixel on the screen. Each pixel has a location based upon an x-y coordination plane. Each pixel moving from left to right is one more in the x horizontal direction. Each pixel moving from top to bottom is one more in the y vertical direction.  If a pixel location matches the location where we want to draw a line, then instead of drawing the actual pixel value, we replace it with a red, green, blue, or yellow color. 

For example, let's draw a horizontal blue line that is 6 pixels wide starting at pixel y location 256. So our logic would say that for any pixel with its y value equal to 256, 257, 258, 259, 260, or 261, we are going to make the pixel blue. If we were to do this on the RGB color mixing image from the RGB_Filter, it would look like :

<img src="data/LineHorizontalColor.png"/>

We could also draw a horizontal red line that is 6 pixels wide sarting at pixel x location 128. Our logic would say that for any pixel with its x value equal to 128, 129, 130, 131, 132, or 133, we are going to make the pixel red. If we were to do this on the RGB color mixing image from the RGB_Filter, it would look like :

<img src="data/LineVerticalColor.png"/>

This notebook will use a video filter that will draw four lines on the screen using the same general idea described above. These four lines will form a box ontop of the image.

#### 1. Download base overlay to the board
Ensure that the camera is not connected to the board. Run the following script to provide the PYNQ with its base overlay.

In [ ]:
from pynq.drivers.video import HDMI
from pynq import Bitstream_Part
from pynq.board import Register
from pynq import Overlay

Overlay("demo.bit").download()

#### 2. Connect camera
Physically connect the camera to the HDMI-in port of the PYNQ. Run the following code to instruct the PYNQ to capture the video from the camera and to begin streaming video to your monitor (connected to the HDMI-out port).

In [ ]:
hdmi_in = HDMI('in')
hdmi_out = HDMI('out', frame_list=hdmi_in.frame_list)
hdmi_out.mode(2)
hdmi_out.start()
hdmi_in.start()

#### 3. Program board with Box Filter
Run the following script to download the RGB Filter to the PYNQ. This will allow us to modify the colors of the video stream.

In [ ]:
Bitstream_Part("lines_p.bit").download()

#### 4. Create a user interface
We will communicate with the filter using a nice user interface. Run the following code to activate that interface.

In [ ]:
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Label

words = ['HDMI Reset']
items = [Button(description=w) for w in words]


def on_hdmi_clicked(b):
    hdmi_out.stop()
    hdmi_in.stop()
    hdmi_out.start()
    hdmi_in.start()

R0=Register(0)
R1=Register(1)
R2=Register(2)
R3=Register(3)
R4=Register(4)

R0.write(200)
R1.write(1020)
R2.write(175)
R3.write(620)
R4.write(3)

R0_s = widgets.IntSlider(
    value=200,
    min=0,
    max=1919,
    step=1,
    description='X1:',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=True,
    readout_format='i',
    slider_color='red'
)
R1_s = widgets.IntSlider(
    value=1020,
    min=0,
    max=1919,
    step=1,
    description='X2:',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=True,
    readout_format='i',
    slider_color='green'
)
R2_s = widgets.IntSlider(
    value=175,
    min=0,
    max=1079,
    step=1,
    description='Y1:',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=True,
    readout_format='i',
    slider_color='blue'
)
R3_s = widgets.IntSlider(
    value=620,
    min=0,
    max=1079,
    step=1,
    description='Y2:',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=True,
    readout_format='i',
    slider_color='yellow'
)
R4_s = widgets.IntSlider(
    value=3,
    min=0,
    max=500,
    step=1,
    description='Width:',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=True,
    readout_format='i',
    slider_color='purple'
)

def update_r0(*args):
    R0.write(R0_s.value)
R0_s.observe(update_r0, 'value')
def update_r1(*args):
    R1.write(R1_s.value)
R1_s.observe(update_r1, 'value')
def update_r2(*args):
    R2.write(R2_s.value)
R2_s.observe(update_r2, 'value')
def update_r3(*args):
    R3.write(R3_s.value)
R3_s.observe(update_r3, 'value')
def update_r4(*args):
    R4.write(R4_s.value)
R4_s.observe(update_r4, 'value')

items[0].on_click(on_hdmi_clicked)

widgets.HBox([VBox([items[0]]),R0_s,R1_s,R2_s,R3_s,R4_s])

#### 5. Exploration
Feel free to play with the sliders above. As you move the sliders up and down, the line on the image will move left and right (for X1 and X2) or up and down (for Y1 and Y2). The slider labeled "Width" will increase the width of the lines on the image.

#### 6. Clean up
When you are done playing with the RGB filter, run the following code to stop the video stream

In [ ]:
hdmi_out.stop()
hdmi_in.stop()
del hdmi_out
del hdmi_in